# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Get total number of rows 
#print(len(full_data_rows_list))
# List of event data rows
#print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating workspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### First Query

In [8]:
# Drop table in case it exists
session.execute("DROP TABLE IF EXISTS songs_by_user_sessions")

In [9]:
# Creating table
# In `songs_by_user_sessions` column sessionId was used as used as partition key and itemInSession as clustering key.
# This configuration allows us to filter the `sessionId` column and makes easier to locate itemInSession, since it defines
# how the rows are sorted.
query = "CREATE TABLE IF NOT EXISTS songs_by_user_sessions"
query = query + "(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY(sessionId, itemInSession))"

session.execute(query)                    

In [10]:
# Getting information from the .csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user_sessions (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = "SELECT sessionId, itemInSession, artist, song, length FROM songs_by_user_sessions WHERE sessionid = 338 AND iteminsession = 4"
rows = session.execute(query)

for row in rows:
    print("| ",str(row.sessionid)," | ", row.iteminsession," | ", row.artist," | ", row.song," | ", row.length," |")

|  338  |  4  |  Faithless  |  Music Matters (Mark Knight Dub)  |  495.30731201171875  |


### Second Query

#### Creating table

In [12]:
session.execute("DROP TABLE IF EXISTS artists_by_user_sessions")

In [13]:
# In `artists_by_user_session` we need to filter and display rows in a specific order. To better do this
# a composite key was used, so it makes finding the rows faster using `userid` and `sessionid` columns. 
# The `iteminsession` as clustering key alows us to sort the information in the order requested.
query = "CREATE TABLE IF NOT EXISTS artists_by_user_sessions"
query = query + "(userid TEXT,  sessionid TEXT, iteminsession TEXT, firstname TEXT, lastname TEXT, artist TEXT, song TEXT, PRIMARY KEY((userid, sessionid), iteminsession))"

session.execute(query)

#### Inserting data into the table

In [14]:
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artists_by_user_sessions (userid, sessionid, iteminsession, firstname, lastname, artist, song)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[10], line[8], line[3], line[1], line[4], line[0], line[9]))

#### Checking Query

In [15]:
query = "SELECT artist, song, iteminsession, firstname, lastname FROM artists_by_user_sessions WHERE userid = '10' AND sessionid = '182'"
rows = session.execute(query)
for row in rows:
    print("| ", row.artist, " | ", row.song, " | ",row.iteminsession, " | ", row.firstname, " | ",row.lastname )

|  Down To The Bone  |  Keep On Keepin' On  |  0  |  Sylvie  |  Cruz
|  Three Drives  |  Greece 2000  |  1  |  Sylvie  |  Cruz
|  Sebastien Tellier  |  Kilometer  |  2  |  Sylvie  |  Cruz
|  Lonnie Gordon  |  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  |  3  |  Sylvie  |  Cruz


### Third Query

#### Creating Table

In [16]:
session.execute("DROP TABLE IF EXISTS users_by_song")

In [17]:
# In the `users_by_song` table we understand that even though we need to filter the table by song
# (primary key) that, alone, is not efficient to retrieve data we need. So, we need to set a clustering 
# key to retrieve the 'firstname' and 'lastname' columns. Since the userid information is related to these
# two, using it as a clustering key seems to be a good idea.
query = "CREATE TABLE IF NOT EXISTS users_by_song"
query = query + "(song TEXT, userid INT, firstname TEXT, lastname TEXT, PRIMARY KEY (song, userid))"

session.execute(query)                    

#### Inserting data into table

In [18]:
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, userid, firstname, lastname)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

#### Checking Query

In [19]:
query = "SELECT firstname, lastname, song FROM users_by_song WHERE song='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print('| ',row.firstname, ' | ', row.lastname, ' | ', row.song, ' |')

|  Jacqueline  |  Lynch  |  All Hands Against His Own  |
|  Tegan  |  Levine  |  All Hands Against His Own  |
|  Sara  |  Johnson  |  All Hands Against His Own  |


### Drop the tables before closing out the sessions

In [20]:
session.execute("DROP TABLE IF EXISTS songs_by_user_sessions")

In [21]:
session.execute("DROP TABLE IF EXISTS artists_by_user_sessions")

In [22]:
session.execute("DROP TABLE IF EXISTS users_by_song")

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()